In [42]:
# Required library imports
import csv
import cPickle as pickle
import nltk
import nltk.corpus
import nltk.tokenize.punkt
import nltk.data
import string
from collections import OrderedDict, Counter
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

#File imports
filepath = "C:/Users/Sven/Google Drive/Sven vd Beukel (Msc)/datasets/"
proverbs = pickle.load(open("%sproverbs.pickle"%filepath))
oneliners = pickle.load(open("%shumorous_oneliners.pickle"%filepath))
headlines = pickle.load(open("%sreuters_headlines.pickle"%filepath))
wikipedia = pickle.load(open("%swiki_sentences.pickle"%filepath))

# Get default English stopwords and extend with punctuation
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(string.punctuation)
stopwords.append('')

# Functions
def measure_overlap_bows(a, b, threshold=0.05): 
    """Check if a and b are matches."""
    tokens_a = [token.lower().strip(string.punctuation) for token in nltk.word_tokenize(a)\
                if token.lower().strip(string.punctuation) not in stopwords]
    tokens_b = [token.lower().strip(string.punctuation) for token in nltk.word_tokenize(b)\
                if token.lower().strip(string.punctuation) not in stopwords]
    # Calculate Jaccard similarity
    ratio = len(set(tokens_a).intersection(tokens_b)) / float(len(set(tokens_a).union(tokens_b)))
    return (ratio >= threshold)

def filter_similar_definitions(definition_list):
    remove_items = []
    for i in range(len(definition_list)+1):
        for definition in range(i+1,len(definition_list)):
            # Remove definitions with high path similarity
            #similarity = definition_list[i][0].path_similarity(definition_list[definition][0])
            #if similarity >= 0.30:
                #print "'%s' and '%s' have a similarity of %f"%(definitions[k][1],definitions[s][1],similarity)
                #remove_items.append(definition_list[definition][0])
            # Remove items with word overlap in definitions
            if measure_overlap_bows(definition_list[i][1], definition_list[definition][1]) == True:
                remove_items.append(definition_list[definition][0])
    remove_items = list(OrderedDict.fromkeys(remove_items))
    distinct_defs = []

    for item in definition_list:
        if item[0] not in remove_items:
            distinct_defs.append(item)
    return distinct_defs

def check_homonymy(word):
    definitions = []
    for i in wn.synsets(word,'n'):
        definitions += [[i, i.definition()]]
    definitions = filter_similar_definitions(definitions)
    if len(definitions) >= 2:
        return True
    
def split_contractions(sentence):
    sentence = sentence.split(' ')
    processed_words = []
    for word in sentence:
        if "'s" in word:
            word = word.split("'")
            first_word = word[0]
            second_word = 'is'
            processed_words.append(first_word)
            processed_words.append(second_word)
        elif "'re" in word:
            word = word.split("'")
            first_word = word[0]
            second_word = 'are'
            processed_words.append(first_word)
            processed_words.append(second_word)
        elif "'ll" in word:
            word = word.split("'")
            first_word = word[0]
            second_word = 'will'
            processed_words.append(first_word)
            processed_words.append(second_word)
        else:
            processed_words.append(word)
    separator = ' '
    processed_sentence = separator.join(processed_words)
    return processed_sentence
    
def pos_tag_sentence(sentence):
    sentence = split_contractions(sentence)
    pos_tagger = nltk.word_tokenize(sentence)
    sent = [word.lower().strip(string.punctuation) for word in pos_tagger]
    tagged_sentence = pos_tag(' '.join(sent).split())
    return tagged_sentence

def find_nouns(sentence):
    pos_tagged_sentence = pos_tag_sentence(sentence) 
    nouns = [word.lower().strip(string.punctuation) for word,pos in pos_tagged_sentence\
             if pos == 'NN' or pos == 'NNS']# or pos == 'NNP' or pos == 'NNPS']
    return nouns

def find_homonyms(sentence):
    nouns = find_nouns(sentence)
    homonym_nouns = [noun for noun in nouns if check_homonymy(noun) == True]
    return homonym_nouns

def find_homonyms_test(sentence,nouns):
    homonym_nouns = [noun for noun in nouns if check_homonymy(noun) == True]
    return homonym_nouns

print 'Executed succesfully'

Executed succesfully


In [43]:
# File import:
with open('one_liners_random_sample_nouns+homonyms.csv', 'r') as t: 
    spamreader = csv.reader(t, delimiter=';', quotechar='|')
    sentences = []
    annotated_nouns = []
    annotated_homonyms = []
    for row in spamreader:
        sentences.append(row[0].decode('ascii', 'ignore'))
        if len(row) >= 2:
            annotated_nouns.append(row[1].split(','))
        if len(row) >= 3:
            annotated_homonyms.append(row[2].split(','))

# Strip away spaces
for i in range(len(annotated_nouns)):
    words = []
    for word in annotated_nouns[i]:
        words.append(word.lower().strip())
    annotated_nouns[i] = words
    
for i in range(len(annotated_homonyms)):
    words = []
    for h in annotated_homonyms[i]:
        words.append(h.lower().strip())
    words = filter(None, words)
    annotated_homonyms[i] = words

# Compare automatic recognized nouns and homonyms to annotated ones
fp_nouns = []
fn_nouns = []
correct_nouns = []
fp_homonyms = []
fn_homonyms = []
correct_homonyms = []

for i in range(len(sentences)):
    found_homonyms = find_homonyms_test(sentences[i],annotated_nouns[i])
    found_nouns = find_nouns(sentences[i])

    for noun in found_nouns:
        if noun in annotated_nouns[i]:
            correct_nouns.append(noun)
        else:
            fp_nouns.append(noun)
            #print '%s was incorrectly classified as noun'%noun
            
    for noun in annotated_nouns[i]:
        if noun not in found_nouns:
            fn_nouns.append(noun)
            #print '%s was incorrectly dismissed'%noun
            
    for h in found_homonyms:
        if h in annotated_homonyms[i]:
            correct_homonyms.append(h)
        else:
            fp_homonyms.append(h)
            #print '%s was incorrectly classified as homonym'%h
            
    for h in annotated_homonyms[i]:
        if h not in found_homonyms:
            fn_homonyms.append(h)
            #print '%s was incorrectly dismissed'%h

# Calculate all evaluation measures
# Nouns
accuracy_nouns = 1-(1.0*(len(fp_nouns)+ len(fn_nouns))/(len(fp_nouns)+ len(fn_nouns)+ len(correct_nouns)))
precision_nouns = 1.0 * len(correct_nouns)/ (len(correct_nouns)+len(fp_nouns))
recall_nouns = 1.0 * len(correct_nouns)/ (len(correct_nouns)+len(fn_nouns))
f_nouns = 2.0 * ((precision_nouns * recall_nouns) / (precision_nouns + recall_nouns))

# Homonyms
accuracy_homonyms = 1-(1.0*(len(fp_homonyms)+ len(fn_homonyms))/(len(fp_homonyms)+ len(fn_homonyms)+ len(correct_homonyms)))
precision_homonyms = 1.0 * len(correct_homonyms)/ (len(correct_homonyms)+len(fp_homonyms))
recall_homonyms = 1.0 * len(correct_homonyms)/ (len(correct_homonyms)+len(fn_homonyms))
f_homonyms = 2.0 * ((precision_homonyms * recall_homonyms) / (precision_homonyms + recall_homonyms))

print 'Done'

Done


In [44]:
print 'Nouns: \t\tFP - %d\tFN - %d\tPrecision: %.4f\tRecall: %.4f\tAccuracy: %.4f\tF-measure: %.4f'\
%(len(fp_nouns), len(fn_nouns),precision_nouns, recall_nouns, accuracy_nouns, f_nouns)
print 'Homonyms: \tFP - %d\tFN - %d\tPrecision: %.4f\tRecall: %.4f\tAccuracy: %.4f\tF-measure: %.4f'\
%(len(fp_homonyms), len(fn_homonyms), precision_homonyms, recall_homonyms, accuracy_homonyms, f_homonyms)
#print len(correct_nouns)
#print len(correct_homonyms)

Nouns: 		FP - 45	FN - 29	Precision: 0.8520	Recall: 0.8993	Accuracy: 0.7778	F-measure: 0.8750
Homonyms: 	FP - 37	FN - 5	Precision: 0.7886	Recall: 0.9650	Accuracy: 0.7667	F-measure: 0.8679


In [31]:
# Evaluate noun performance:
print "The following words were incorrectly classified as noun:"
print fp_nouns
print "The following nouns were incorrectly dismissed:"
print fn_nouns

The following words were incorrectly classified as noun:
[u'spend', u'ill', u'red', u'ex', u'fine', u'robbers', u'fancy', u'backwards', u'blonde', u'worth', u'doing', u'secret', u'fuck', u'none', u'while', u'farts', u'forget', u'wet', u'lets', u'black', u'shit', u'whatever', u'becomes', u'happens', u'ta', u'dig', u'awkward', u'awkward', u'awkward', u'counts', u'hot', u'help', u'leave', u'o', u'shea', u'stands', u'oooooh', u'better', u'random', u'brace', u'stolen', u'touches', u'say', u'pity', u'yes']
The following nouns were incorrectly dismissed:
['green', 'google', 'google', 'learning', 'napoleon', 'coat', 'library', 'librarian', 'half-brother', '', '%', '%', 'sweden', 'ring', 'ting', 'seed', "o'shea", 'cannoli', 'odeon', 'cinemas', 'funeral', 's_e_e', 'fish', 'bed', 'red', 'whisky', 'dad', 'mr.t', 'dave']


In [28]:
print "The following words were incorrectly classified as homonym:"
print fp_homonyms
print "The following homonyms were incorrectly dismissed:"
print fn_homonyms

The following words were incorrectly classified as homonym:
['grave', 'dress', 'marriage', 'suicide', 'christmas', 'suspense', 'suicide', 'fun', 'women', 'teachers', 'problems', 'marriage', 'license', 'thought', 'women', 'interview', 'strengths', 'example', 'teacher', 'student', 'minimum', 'maximum', 'seed', 'rick', 'cinemas', 'knife', 'fight', 'clowns', 'god', 'skin', 'water', 'bone', 'gypsy', 'pushover', 'disaster']
The following homonyms were incorrectly dismissed:
['scotch', 'keyboard', 'hanger', 'chick', 'tiles']


In [ ]:
#from ark_tweet_nlp_python_master import CMUTweetTagger as CM
#results= CM.runtagger_parse(['example tweet 1', 'example tweet 2'])
#print results
'''from nltk.tag import StanfordPOSTagger
import os
java_path = "C:/Program Files (x86)/Java/jdk1.8.0_91" # replace this
os.environ['JAVAHOME'] = java_path
print os.environ['JAVAHOME']
st = StanfordPOSTagger('C:/Python27/Lib/stanford-postagger-2015-12-09/models/english-bidirectional-distsim.tagger',\
                       'C:/Python27/Lib/stanford-postagger-2015-12-09/stanford-postagger.jar') 
st.tag('What is the airspeed of an unladen swallow ?'.split())'''

In [ ]:
# Archive for barely used code that may come in handy someday
# Create Bag of Words for each dataset, ignoring most common english stopwords
BoW_proverbs = [token.lower().strip(string.punctuation) for sentence in \
               proverbs for token in nltk.word_tokenize(sentence)\
                if token.lower().strip(string.punctuation) not in stopwords]

BoW_oneliners = [token.lower().strip(string.punctuation) for sentence in \
               oneliners for token in nltk.word_tokenize(sentence)\
                if token.lower().strip(string.punctuation) not in stopwords]

BoW_headlines = [token.lower().strip(string.punctuation) for sentence in \
               headlines for token in nltk.word_tokenize(sentence)\
                if token.lower().strip(string.punctuation) not in stopwords]
# Most frequent words
print "The most common words in the Proverbs dataset are:"
print BoW_proverbs_word_freq.most_common()[:200]
print "The most common words in the Oneliners dataset are:"
print BoW_oneliners_word_freq.most_common()[:20]
print "The most common words in the Headlines dataset are:"
print BoW_headlines_word_freq.most_common()[:20]

# Compute lexical diversity
def lexical_diversity(tokens):
    return 1.0*len(set(tokens))/len(tokens)

# Compute the average number of words per tweet
def average_words(sentences):
    total_words = sum([len(s.split()) for s in sentences ])
    return 1.0*total_words/len(sentences)

def average_word_length(tokens):
    total_word_length = sum([ len(w) for w in tokens ])
    return 1.0*total_word_length/len(tokens)

def tokens_per_sentence(tokens, sentences):
    total_word_length = sum([ len(w) for w in tokens ])
    return 1.0*total_word_length/len(sentences)
print "Proverbs -  Lexical Diversity: %f \t\t AVG Word count: %f \t AVG Word Length: %f"\
%(lexical_diversity(BoW_proverbs), average_words(proverbs), average_word_length(BoW_proverbs))
print "Oneliners - Lexical Diversity: %f \t AVG Word count: %f \t AVG Word Length: %f"\
%(lexical_diversity(BoW_oneliners), average_words(oneliners), average_word_length(BoW_oneliners))
print "Headlines - Lexical Diversity: %f \t AVG Word count: %f \t AVG Word Length: %f"\
%(lexical_diversity(BoW_headlines), average_words(headlines), average_word_length(BoW_headlines))

print tokens_per_sentence(BoW_proverbs,proverbs)
print tokens_per_sentence(BoW_oneliners,oneliners)
print tokens_per_sentence(BoW_headlines,headlines)


for word in [BoW_proverbs]:
    BoW_proverbs_word_freq = Counter(word)

for word in [BoW_oneliners]:
    BoW_oneliners_word_freq = Counter(word)

for word in [BoW_headlines]:
    BoW_headlines_word_freq = Counter(word)
    
#Named Entity Recognition
'''tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

#entity_names = []
#for tree in chunked_sentences:
    # Print results per sentence
    #print extract_entity_names(tree)

    #entity_names.extend(extract_entity_names(tree))

# Print all entity names
#print entity_names

#filepath = "C:/Users/Sven/Google Drive/Sven vd Beukel (Msc)/datasets/"
#oneliners = pickle.load(open("%shumorous_oneliners.pickle"%filepath))

# indexpositions of random selection of oneliners, currently ununsed:
random_selection = [75, 648, 680, 713, 735, 813, 900, 975, 984, 1062, 1156, 1226, 1302, 1372, 1415, 1491, 1578, 1661, 1663, 1680, 1832,
             1841, 1846, 1888, 1916, 1981, 1993, 1994, 2017, 2034, 2159, 2240, 2275, 2513, 2747, 2756, 2943, 3006, 3078, 3121, 
             3162, 3184, 3187, 3242, 3267, 3505, 3510, 3551, 3620, 3626, 3643, 3649, 3843, 3853, 3858, 3925, 3927, 4002,
             4018, 4030, 4106, 4154, 4155, 4209, 4334, 4416, 4433, 4469, 4510, 4517, 4610, 4771, 4904, 4912, 5058, 5138]
'''

In [ ]:
# Code for extracting most similar wikipedia sentences in comparison with oneliners
unprocessed_wiki_sentences = open('wiki_sentences.txt', 'r')
unprocessed_wiki_sentences = unprocessed_wiki_sentences.readlines()
wiki_sents = []
for s in unprocessed_wiki_sentences:
    s = s.split('\n')
    for i in s:
         if i == '':
            s.remove(i)
    wiki_sents.append(s[0])

similar_sentences = []
sentences_processed = 0
for sentence in wiki_sents:
    sentence = sentence.split(' ')
    similar_words = 0
    for word in sentence:
        for w in BoW_proverbs_word_freq.most_common()[:200]:
            if word.lower() == w[0]:
                similar_words +=1
    sentence_similarity = (1.0*similar_words)/len(sentence)
    if sentence_similarity>0.05:
        similar_sentences.append((sentence,sentence_similarity))
    sentences_processed += 1
    if sentences_processed%2000 == 0:
        print "processed %s sentences"%sentences_processed
similar_sentences2 = similar_sentences
def getKey(item):
    return item[1]
similar_sentences2 = sorted(similar_sentences2, key=getKey, reverse = True)

most_similar_sents = []
for s in similar_sentences2[0:5252]:
    most_similar_sents.append(" ".join(s[0]))
    
# Store away the most similar wikipedia sentences
f = open("wiki_sents.pickle", "wb")
pickle.dump(most_similar_sents, f)
f.close()